In [1]:
import tensorflow as tf
import keras
from keras import backend as K
from tensorflow.python.client import device_lib
import pandas as pd
import re
import numpy as np

import requests
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU, LSTM
from keras.layers import Embedding
from timeit import default_timer as timer
import math
import random
from collections import defaultdict
from keras.models import model_from_json
import h5py
import numpy as np
import sentencepiece as spm
from sklearn.model_selection import train_test_split
import sentencepiece as sp
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras import backend as K
import scipy
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from keras.callbacks import EarlyStopping
from sklearn.metrics import log_loss, make_scorer
import time
from sklearn.model_selection import StratifiedKFold
from keras.regularizers import l1
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
import os


import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

random.seed(11)
print(tf.__version__)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))

2.4.1
Found GPU at: /device:GPU:0


In [2]:
sp = spm.SentencePieceProcessor(model_file='book.model')
with open("testbook.txt") as handle:
    sentences =  [l.strip() for l in handle.readlines()]
    
v = 3000


In [5]:
# Load text into sentences, vocab size of 8k, maybe making it higher will increase performance
sp = spm.SentencePieceProcessor(model_file='brown.model')

with open("brownsent.txt") as handle:
    sentences =  [l.strip() for l in handle.readlines()]
    
v = 8000

In [6]:
# remove very long and very short sentences
x = []

for a in sentences:
    if len(a.split()) > 2 and len(a.split()) < 70:
        x.append(a)

In [7]:
# Encode sentences
X = []
X1 = []
y1 = []
for sentence in x:
    X.append(sp.encode(sentence))
    
for sentence in X:
    y1.append(sentence[-1])
    X1.append(sentence[0:-1])
    
y = to_categorical(y1, num_classes=v)

In [8]:
# Max sent length for padding
max1 = 0
for i in x:
    if len(i) > max1:
        max1 = len(i)
print(max1)

453


In [9]:
# Pad sequences
X2 = []
for XD in X1:
    X2.append(pad_sequences([XD], maxlen=max1, padding='pre'))
    
X2 = np.asarray(X2)
X2

X3 = []
for i in X2:
    X3.append(i[0])
    
    
X3 = np.asarray(X3)
X = X3
X

array([[   0,    0,    0, ...,    4,   29, 4852],
       [   0,    0,    0, ...,   43,   12, 1234],
       [   0,    0,    0, ...,   36,  378,   92],
       ...,
       [   0,    0,    0, ...,    3,  340,    3],
       [   0,    0,    0, ...,    5, 3931, 6043],
       [   0,    0,    0, ...,  920, 1291, 2127]])

In [10]:
# Shuffle data
c = list(zip(X, y))

random.shuffle(c)

X, y = zip(*c)

X = np.asarray(X)
y = np.asarray(y)
X

array([[   0,    0,    0, ...,   46,  274,   12],
       [   0,    0,    0, ...,  100,   75,   83],
       [   0,    0,    0, ..., 1947,   27,    5],
       ...,
       [   0,    0,    0, ..., 1344,   12,   44],
       [   0,    0,    0, ..., 2329,   85,  214],
       [   0,    0,    0, ..., 2854,   76,   18]])

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [12]:
X_train

array([[   0,    0,    0, ...,   38,    3,  184],
       [   0,    0,    0, ...,    5,  329,    8],
       [   0,    0,    0, ...,    3,    6,    5],
       ...,
       [   0,    0,    0, ..., 6629, 1243,  402],
       [   0,    0,    0, ...,    4,  104, 5854],
       [   0,    0,    0, ...,    4,  217,    9]])

In [13]:
np.set_printoptions(precision=None, suppress=True)
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, restore_best_weights=True)

In [ ]:
# My own grid search implementation
def grid_search(epochs, batches, dropout, lr):
    # Meshgrid creates arrays of every possible combinations
    grid = np.array(np.meshgrid(batches,dropout,lr)).T.reshape(-1, 3)
    
    # for every combination in grid...
    for element in grid:
        
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=str(element) + "/modelcheckP.h5",
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            save_best_only=True)
        
        # Create directory with the name of the parameters
        try:
            os.mkdir(str(element))
        except OSError:
            print("Dir exists already")
            
            
        # Create the model
        model = Sequential()
        optimizer = keras.optimizers.Adam(lr=element[2])
        model.add(Embedding(v, 512, input_length = max1, mask_zero=True))
        model.add(LSTM(512, return_sequences=True))
        model.add(Dropout(element[1]))
        model.add(LSTM(512))
        model.add(Dense(v, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer=optimizer)
        
        
        # Fit the model, right now the amount of epochs is just fixed to 10. Callbacks with
        # modelcheckpoint and early stopping.
        print("Now fitting parameters: " + str(element))
        start = time.time()
        model.fit(X_train, y_train, epochs=10, verbose=1, batch_size=int(element[0]), callbacks=[early_stopping, model_checkpoint_callback], validation_data=(X_test, y_test))
        end = time.time()
        print("Time:")
        print(end - start)

        
        # Save model and weights to directory
        model_json = model.to_json()
        with open(str(element) + "/model.json", "w") as json_file:
            json_file.write(model_json)
        json_file.close()
        model.save_weights(str(element) + "/weights.h5")
        

# grid_search(epochs, batches, dropout, lr), possible to add more parameters.
grid_search(5,[256],[0.2],[0.02])


Now fitting parameters: [256.     0.2    0.02]
Epoch 1/10


# This is the kears grid search, which takes about as long

In [17]:
def create_model(optimizer='adam', dropout=0.1):
    K.clear_session()
    model = Sequential()
    model.add(Embedding(v, 200, input_length = max1, mask_zero=True))y

    model.add(LSTM(512, return_sequences=True))

    model.add(LSTM(512))

    # output layer
    model.add(Dense(v, activation='softmax'))
    model.add(Dropout(dropout))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    return model

# Create param_grid which is a dictionary of parameters to test
# only using single parameters right now to speed up the proces
model = KerasClassifier(build_fn=create_model, verbose=0)
# Add things to these lists to add parameters to be tested
epochs = [10]
batches = [128]
optimizers = ['adam']
dropout= [0.3,0.4,0.5,0.6]
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=6, restore_best_weights=True)
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, dropout=dropout)
grid = GridSearchCV(estimator=model, param_grid=param_grid, return_train_score=True, cv = 2, scoring=scorer)

In [18]:
start = time.time()
grid_result = grid.fit(X_train, y_train, callbacks=[early_stopping], validation_data=(X_test, y_test), verbose=1)
end = time.time()
print(end - start)

Epoch 1/10
161/161 [==============================] - 822s 5s/step - loss: 9.9564 - val_loss: 7.0060
Epoch 2/10
161/161 [==============================] - 822s 5s/step - loss: 9.2942 - val_loss: 6.8697
Epoch 3/10
161/161 [==============================] - 811s 5s/step - loss: 9.1034 - val_loss: 6.8238
Epoch 4/10
161/161 [==============================] - 813s 5s/step - loss: 9.0396 - val_loss: 6.7569
Epoch 5/10
161/161 [==============================] - 809s 5s/step - loss: 8.9895 - val_loss: 6.7376
Epoch 6/10
161/161 [==============================] - 811s 5s/step - loss: 8.8552 - val_loss: 6.7195
Epoch 7/10
161/161 [==============================] - 808s 5s/step - loss: 8.7248 - val_loss: 6.7132
Epoch 8/10
161/161 [==============================] - 808s 5s/step - loss: 8.6020 - val_loss: 6.7449
Epoch 9/10
161/161 [==============================] - 815s 5s/step - loss: 8.5140 - val_loss: 6.6995
Epoch 10/10
161/161 [==============================] - 813s 5s/step - loss: 8.4052 - val_lo

KeyboardInterrupt: 

In [19]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

model = grid_result.best_estimator_

NameError: name 'grid_result' is not defined

In [15]:
model = grid_result.best_estimator_

Make predictions using this function

In [25]:
def generate_words(model, text):
    # create word embeddings


    padded_words = pad_sequences([text], maxlen=max1, padding='pre')
    # predict next word
    prediction = model.predict_proba(padded_words)[0][0:20]
    a = int(np.argmax(model.predict_proba(padded_words)[0]))
    print(a)
    print(sp.decode([a])) 
        

In [34]:
XD = []
county = 7

for i in X_train[county][np.nonzero(X_train[county])[0][0]:]:
    XD.append(int(i))
print(sp.decode(XD))

generate_words(model, XD)

Martha, who was a poor apprentice at a milliner's, then told them what kind of work she had to do, and how many hours she worked at a stretch, and how she meant to lie abed to-morrow morning for a good long rest; to-morrow being a holiday she passed at
14
it


In [28]:
with open('brownsent.txt', 'r') as file:
    data = file.read().replace('\n', '')

data = data.split()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [29]:
def baseline_perplex(data):
    distribution = defaultdict(int)
    for word in data:
        distribution[word] += 1
        
    probdistribution = []   
    sumD = sum(distribution.values())
    for num in list(distribution.values()):
        probdistribution.append(num/sumD)
        
    count = 0
    for key in distribution.keys():
        distribution[key] = probdistribution[count]
        count += 1

    chance = 0
    for i in range(len(y_train)):
        chance += math.log2(distribution.get(y_train[i], (1/sumD)))
    
    return 2**((-1/len(y_train))* chance)


baseline_perplex(X_train)

1582.812467959088

In [18]:
def perplexity(Xp, yp, model):

    distributionList = model.predict_proba(Xp)
    chance = 0

    for i in range(len(distributionList)):
        chance += math.log2(distributionList[i][np.nonzero(yp[i])[0][0]])

    return 2**((-1/len(Xp))* chance)

start = time.time()
print(perplexity(X_test, y_test, model))
end = time.time()
print(end - start)

NameError: name 'model' is not defined